In [1]:
import sys
import time
import zmq
from PyQt5 import sip
import pandas as pd


from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QSize, Qt
from PyQt5.QtWidgets import *

# 全局变量

In [2]:
AcctNameDict = {}

In [3]:
priceType ={}
priceType['1'] = "AnyPrice"
priceType['2'] = "LimitPrice"
priceType['u'] = "Unknown"
priceType

{'1': 'AnyPrice', '2': 'LimitPrice', 'u': 'Unknown'}

In [4]:
directionType ={}
directionType['0'] = "Buy"
directionType['1'] = "Sell"
directionType['u'] = "Unknown"
directionType

{'0': 'Buy', '1': 'Sell', 'u': 'Unknown'}

In [5]:
OpenCloseFlag ={}
OpenCloseFlag['0'] = "Open"
OpenCloseFlag['1'] = "Close"
OpenCloseFlag['3'] = "CloseToday"
OpenCloseFlag['4'] = "CloseYesterday"
OpenCloseFlag['u'] = "Unknown"
OpenCloseFlag

{'0': 'Open',
 '1': 'Close',
 '3': 'CloseToday',
 '4': 'CloseYesterday',
 'u': 'Unknown'}

In [6]:
hedgeFlag = {}
hedgeFlag['u'] = "Unknown"
hedgeFlag['1'] = "Speculate"
hedgeFlag['2'] = "Arbitrage"
hedgeFlag

{'u': 'Unknown', '1': 'Speculate', '2': 'Arbitrage'}

In [7]:
orderStatus ={}
orderStatus['1'] = "New"
orderStatus['2'] = "Accepted"
orderStatus['3'] = "Rejected"
orderStatus['4'] = "Cancelling"
orderStatus['5'] = "Canceled"
orderStatus['6'] = "PartTradedQueueing"
orderStatus['7'] = "PartTradedNotQueueing"
orderStatus['8'] = "AllTraded"
orderStatus['u'] = "Unknown"
orderStatus

{'1': 'New',
 '2': 'Accepted',
 '3': 'Rejected',
 '4': 'Cancelling',
 '5': 'Canceled',
 '6': 'PartTradedQueueing',
 '7': 'PartTradedNotQueueing',
 '8': 'AllTraded',
 'u': 'Unknown'}

# 相关函数

获取账户 字典

In [8]:
def GetAcctName():
    tb = pd.read_csv("./depend/accountMap.txt",header=None)
    tb.index = tb[0]
    tb = tb[[1]]
    tb.index.name=None
    tb_dict = tb.to_dict()
    tb_dict = tb_dict[1]
    
    return tb_dict
GetAcctName()

{8001: 'TEST-FUTURE1',
 8002: 'TEST-STOCK1',
 8003: 'ALPHA-1-FUT',
 8004: 'ALPHA-1-STK',
 8005: 'ALPHA-2-FUT',
 8006: 'ALPHA-2-STK',
 8007: 'ALPHA-3-FUT',
 8008: 'ALPHA-3-STK',
 8009: 'ALPHA-5-FUT',
 8011: 'TEST-FUTURE2',
 8012: 'TEST-FUTURE3',
 8010: 'ALPHA-SW1-STK',
 8013: 'ALPHA_SW1_FUT',
 8014: 'JJ_STOCK',
 8015: 'SW_FUT_2',
 8016: 'JR7_STOCK',
 8017: 'SW_CREDIT_1'}

# 相关类

##  Request 类

In [9]:
###Cedar Request
class Request(object):
    rqAcct=0;
    rqInstrument="";
    rqBatchID="";
    rqid=0;
    rqDirection='u';
    rqOrderSize=0;
    tradedVol=0;
    tradedAvgPrice=0;
    fillRate=0;
    referencePrice=0;

    def fill_value(msg):
        elems = msg.split(',')
        rqAcct = int(elems[0])
        rqInstrument=elems[1]
        rqBatchID=elems[2]
        rqid=int(elems[3])
        rqDirection=elems[4]
        rqOrderSize=int(elems[5])
        tradedVol=int(elems[6])
        tradedAvgPrice=float(elems[7])
        fillRate=float(elems[8])
        referencePrice=float(elems[9])



## Order 类


In [10]:
###Cedar Order
class Order(object):
        m_OrderRef=0;           #### Order ref no, unique for each order in exchange
        m_RequestID=0;          ##/< Request id, unique for each strategy
        m_OrderSysID="";        ##/< Order sys id, used by CTP, according to CTP documentation, It's quicker to use this id when cancelling an order
        m_ProdID="";            ##/< Product id
        m_ExchID="";            ##/< Exchange id
        m_PriceType='u';        ##/< Price type, refer to \ref PriceTypeGroup
        m_Direction='u';        ##/< Order direction, refer to \ref DirectionGroup
        m_CombOffsetFlag='u';   ##/< Offest flag, refer to \ref OffsetFlagGroup
        m_CombHedgeFlag='u';    ##/< Hedge flag, refer to \ref HedgeFlagGroup
        m_LimitPrice=0;         ##/< Limit price
        m_VolOriginal=0;        ##/< Original volume of this order
        m_VolRemaining=0;       ##/< Remaining volume of this order
        m_VolTraded=0;          ##/< Traded volume of this order, updated by OnRtnOrder
        m_VolConfirmTraded=0;	##/< Confirmed traded volume of this order, updated by OnRtnTrade
        m_Status='u';           ##/< Order status, refer to \ref OrderStatusGroup
        m_ChaseTimes=0;         ##/< The times of chasing this order
        m_SequenceNo=0;         ##/< Order sequence no, updated by exchange

        def fill_value(msg):
            elems = msg.split(',')
            m_OrderRef=int(elems[0])
            m_RequestID=int(elems[1])
            m_OrderSysID=elems[2]
            m_ProdID=elems[3]
            m_ExchID=elems[4]
            m_PriceType=elems[5]
            m_Direction=elems[6]
            m_CombOffsetFlag=elems[7]
            m_CombHedgeFlag=elems[8]
            m_LimitPrice=float(elems[9])
            m_VolOriginal=int(elems[10])
            m_VolRemaining=int(elems[11])
            m_VolTraded=int(elems[12])
            m_VolConfirmTraded=int(elems[13])
            m_Status=elems[14]
            m_ChaseTimes=int(elems[15])
            m_SequenceNo=int(elems[16])



## BatchManaer类

In [11]:
class BatchManager():

    def __init__(self):
        self.BuyValuePerRQID = {}
        self.SellValuePerRQID = {}
        self.BuyTotalValuePerRQID = {}
        self.SellTotalValuePerRQID = {}
        self.AcctID = 0
    def bookAcctID(self,acct):
        self.AcctID = acct

    def bookTradedValue(self,RQID,value):
        if(value > 0):
            self.BuyValuePerRQID[RQID] = value
            #print("BUY: %f"%(self.BuyValuePerRQID[RQID]))
        else:
            self.SellValuePerRQID[RQID] = value
            #print("SELL: %f"%(self.SellValuePerRQID[RQID]))
    def bookTotalValue(self,RQID,value):
        if(value > 0):
            self.BuyTotalValuePerRQID[RQID] = value
            #print("BUY: %f"%(self.BuyValuePerRQID[RQID]))
        else:
            self.SellTotalValuePerRQID[RQID] = value
            #print("SELL: %f"%(self.SellValuePerRQID[RQID]))

    def getBuyNotional(self):
        #print("BUY NOTIONAL : ",self.BuyValuePerRQID.values(),sum(self.BuyValuePerRQID.values()))
        boughtNotional = sum(self.BuyValuePerRQID.values())
        totalBuyNotional  = sum(self.BuyTotalValuePerRQID.values())
        myFillRate     = (boughtNotional / totalBuyNotional) if (totalBuyNotional > 0) else 0
        return [boughtNotional,myFillRate,totalBuyNotional]

    def getSellNotional(self):
        #print("SELL NOTIONAL : ",self.SellValuePerRQID.values(),sum(self.SellValuePerRQID.values()))
        soldNoitional = sum(self.SellValuePerRQID.values())
        totalSellNotional = sum(self.SellTotalValuePerRQID.values())
        myFillRate    = soldNoitional / totalSellNotional if((abs(totalSellNotional)) > 0) else 0
        return [soldNoitional,myFillRate,totalSellNotional]

    def getAcctID(self):
        return self.AcctID;

## updata类

In [12]:
class UpdateData(QtCore.QThread):
    requestChanged = QtCore.pyqtSignal(int, int, str)  # rowIndex, msgType,msg

    def run(self):
        requestChanged = QtCore.pyqtSignal(int, int, str)
        context = zmq.Context()
        sock = context.socket(zmq.SUB)
        sock.setsockopt(zmq.SUBSCRIBE, b"EMS_GUI_SubOda")
        sock.setsockopt(zmq.SUBSCRIBE, b"EMS_GUI_Request")
        sock.setsockopt(zmq.SUBSCRIBE, b"EMS_GUI_Error")
        sock.setsockopt(zmq.HEARTBEAT_IVL,     5000)
        sock.setsockopt(zmq.HEARTBEAT_TIMEOUT, 3000)
        #print("hello")
        sock.connect("tcp://192.168.0.66:15300")
        sock.connect("tcp://117.185.37.175:51336")
		### DOUBLE RQID Problem

        while True:
            msg = sock.recv()
            msgs = msg.decode("ascii").split("|")
            #msgs= msg.split(',')
            #self.dataChanged.emit(2, 2, msgs[0])
            print(msgs[0],msgs[1])
            if(msgs[0] == "EMS_GUI_Request"):
                self.requestChanged.emit(1,1, msgs[1])
                print(msgs[1])
            elif(msgs[0] == "EMS_GUI_SubOda"):
                self.requestChanged.emit(2,2, msgs[1])
                print(msgs[1])
            elif(msgs[0] == "EMS_GUI_Error"):
                self.requestChanged.emit(2,3, msgs[1])
                print(msgs[1])

## Control_sys_Tab

In [15]:
class Control_sys_Tab(QTabWidget):
    def __init__(self,parent=None):
        self.RequestRowKey ={}
        self.OrderRowKey  ={}
        self.BatchRowKey  ={}   #To manage batch row index
        self.ErrorRowKey  ={}   #To manage error row index
        self.BuyBatchValue = {}
        self.SellBatchValue= {}
        self.BatchManagers  ={} # To manage value for each batch
        self.g_CurrRequestRow = 0
        self.g_CurrOrderRow  = 0
        self.g_CurrBatchRow  = 0
        self.g_CurrErrorRow  = 0
        super().__init__(parent)
        
        self.setObjectName("Control_system")
        self.resize(1800,985)
        self.setWindowTitle("实时监控系统")
        
        
        #创建3个选项卡小控件窗口
        self.tab1=QWidget()
        self.tab2=QWidget()
        self.tab3=QWidget()
        
        
        #将三个选项卡添加到顶层窗口中
        self.addTab(self.tab1, "交易界面")
        self.addTab(self.tab2, "PNL展示界面")
        self.addTab(self.tab3, "PNL定时发送")

        #每个选项卡自定义的内容
        self.tab1UI()
        self.tab2UI()
        self.tab3UI()
        
        self.pushButton.clicked.connect(self.slotStart)
    
    def tab1UI(self):
        #设置主布局
        layout=QHBoxLayout()
        
        sec_layout=QFormLayout()
        
        # 创建表格窗口1
        self.tableWidget1 = QtWidgets.QTableWidget()
        self.tableWidget1.setRowCount(5000)
        self.tableWidget1.setColumnCount(11)
        self.tableWidget1.setObjectName("tableWidget")
        self.tableWidget1.setAutoFillBackground(True)
        self.tableWidget1.setHorizontalHeaderLabels(["Acct","Instrument","BatchID","RQID","Direction","OrderSize","TradedVol","AvgPrice","Notional","FillRate","RefPrice"])
        for i in range(0,11):
            self.tableWidget1.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
        #self.tableWidget.item(0, 0).setFont(font)
        
        
        # 表格窗口2
        self.tableWidget2 = QtWidgets.QTableWidget()
        self.tableWidget2.setRowCount(20000)
        self.tableWidget2.setColumnCount(14)
        self.tableWidget2.setObjectName("tableWidget")
        self.tableWidget2.setAutoFillBackground(True)
        self.tableWidget2.setHorizontalHeaderLabels(["OrderRef", "RequestID", "PriceType","Direction","OffsetFlag","HedgeFlag","LimitPrice","VolOriginal","VolRemain","VolTraded","VolConfirmed","Status","OrderSysID","ExchID"])
        for i in range(0,14):
            self.tableWidget2.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
        
        
        # 表格窗口3
        self.tableWidget3 = QtWidgets.QTableWidget()
        #self.tableWidget3.setTextAlignment(Qt.AlignHCenter)
        self.tableWidget3.setRowCount(500)
        self.tableWidget3.setColumnCount(6)
        #self.tableWidget3.setStyleSheet('background-repeat:repeat;')  #font color
        #self.tableWidget3.setStyleSheet('color:darkblue;')  #font color
        #self.tableWidget3.setStyleSheet('text-align:center;')
        #self.tableWidget3.setStyleSheet('vertical-align:super;')
        #self.tableWidget3.setStyleSheet('background-color:lightblue')
        #self.tableWidget3.horizontalHeader().setStyleSheet('QHeaderView::section{background:gray}')

        self.tableWidget3.setObjectName("tableWidget")
        self.tableWidget3.setAutoFillBackground(True)
        self.tableWidget3.setHorizontalHeaderLabels(["BatchID", "AcctName","BuyNotional", "SellNotional","BuyFillRate","SellFillRate"])
        for i in range(0,6):
            self.tableWidget3.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            #self.tableWidget3.
        self.tableWidget3.setColumnWidth(0,150)
        self.tableWidget3.setColumnWidth(1,150)
        self.tableWidget3.setColumnWidth(2,100)
        self.tableWidget3.setColumnWidth(3,100)
        self.tableWidget3.setColumnWidth(4,100)
        self.tableWidget3.setColumnWidth(5,100)
        #self.label = QtWidgets.QLabel(self.centralwidget)
        #self.label.setGeometry(QtCore.QRect(360, 70, 300, 50))
        #self.label.setObjectName("label")
        #self.label.setAutoFillBackground(True)
        #self.label.setAlignment(QtCore.Qt.AlignCenter)
        #self.label.setStyleSheet("border-image:url(images/title.png)")
        
        
        self.pushButton = QtWidgets.QPushButton()
        #self.pushButton.setMaximumWidth(100)
        self.pushButton.setObjectName("pushButton")
        self.pushButton.setText("开始运行")
        
        
        
        # 添加表单2进 子布局
        sec_layout.addWidget(self.tableWidget2)
        
        # 添加表单3进 子布局
        sec_layout.addWidget(self.tableWidget3)
        
        # 添加 按钮 进 子布局
        sec_layout.addWidget(self.pushButton)
        
        # 添加表单1 进主布局，子布局进主布局
        layout.addWidget(self.tableWidget1)
        layout.addLayout(sec_layout)
        

        self.tab1.setLayout(layout)
        
        
        
    def tab2UI(self):
        
        return 
    
    def tab3UI(self):
        
        return 
    
    ##############################################################
#     PyQt5 中的pyQtslot 是python中的decorator，用其可以将一个method 定义为 槽
    
#     槽的传参方式 主要是直接传入一个 函数指针
    
    ##############################################################
    
    
    @QtCore.pyqtSlot()
    def slotStart(self):
        #按钮 暂停使用
        self.pushButton.setEnabled(False) 
        # 开启一个新进程用来 更新数据
        self.update_data_thread = UpdateData(self) 
        self.update_data_thread.requestChanged.connect(self.onRequestChanged)
        # 线程进入 准备阶段
        self.update_data_thread.start()

    @QtCore.pyqtSlot(int, int, str)
    def onRequestChanged(self, row, msgType, text):
        elems = text.split(',')
        #row =0
        column = 0
        if(msgType == 1):
            myKey = self.RequestRowKey.get(elems[2] + elems[3])
            if( myKey== None):
                myKey = self.g_CurrRequestRow
                self.g_CurrRequestRow+=1
                self.RequestRowKey[elems[2] + elems[3]] = myKey           ### CREATE A REQUEST ROW

            for ele in elems:
                it = self.tableWidget1.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget1.setItem(myKey, column, it)
                if(column==0):
                    it.setText(AcctNameDict[int(ele)])
                    #print("What is this: %d, and acctName =  %s"%(int(ele),AcctNameDict[int(ele)]))
                elif(column==4):
                    it.setText(directionType[ele])
                else:
                    it.setText(ele)
                column+=1
            self.tableWidget1.selectRow(myKey)

            myKey = self.BatchRowKey.get(elems[2])
            if( myKey== None):### UPDATE ACCT-BATCH TABLE HERE
                myKey = self.g_CurrBatchRow
                self.g_CurrBatchRow+=1
                self.BatchRowKey[elems[2]] = myKey
                self.BatchManagers[elems[2]] = BatchManager()                        ### CREATE A BATCH MANAGER
                self.BatchManagers[elems[2]].bookAcctID(int(elems[0]))
            #print("what is the direction code: %s"%(elems[4]))
            myTradeDirection = 1.0 if (int(elems[4]) == 0) else -1.0
            self.BatchManagers[elems[2]].bookTotalValue(elems[3],float(elems[5])*float(elems[10])*myTradeDirection)
            self.BatchManagers[elems[2]].bookTradedValue(elems[3],float(elems[8]))   ### need to change here
            print(elems[3],float(elems[8]))

            ### FILL INFORMATION IN THIS ROW
            ###print("GET NOTIONA:",str(self.BatchManagers[elems[2]].getBuyNotional()),str(self.BatchManagers[elems[2]].getSellNotional()))
            thisAcctID = self.BatchManagers[elems[2]].getAcctID()
            thisAcctIDStr = AcctNameDict[thisAcctID]
            vals1      = self.BatchManagers[elems[2]].getBuyNotional()
            vals2      = self.BatchManagers[elems[2]].getSellNotional()
            vals       = [elems[2],thisAcctIDStr,"{:,.2f}".format(vals1[0]),"{:,.2f}".format(vals2[0]),"{:.2%}".format(vals1[1]),"{:.2%}".format(vals2[1])]
            for column in range(0,6):
                it = self.tableWidget3.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget3.setItem(myKey, column, it)
                it.setText(vals[column])
            self.tableWidget3.selectRow(myKey)

        elif(msgType == 2):
            myKey = self.OrderRowKey.get(elems[0] + elems[1] + elems[12])
            if( myKey== None):
                myKey = self.g_CurrOrderRow
                self.g_CurrOrderRow+=1
                self.OrderRowKey[elems[0] + elems[1] + elems[12]] = myKey
            for ele in elems:
                it = self.tableWidget2.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget2.setItem(myKey, column, it)# can be done in dictionary of dictionary way, better
                if(column==3):
                    it.setText(directionType[ele])
                elif(column==4):
                    it.setText(OpenCloseFlag[ele])
                elif(column==5):
                    it.setText(hedgeFlag[ele])
                elif(column==11):
                    it.setText(orderStatus[ele])
                else:
                    it.setText(ele)
                column+=1
            self.tableWidget2.selectRow(myKey)
			
        elif(msgType == 3):
            myKey = self.ErrorRowKey.get(elems[0] + elems[2])
            if( myKey== None):
                myKey = self.g_CurrErrorRow
                self.g_CurrErrorRow += 1
                self.ErrorRowKey[elems[0] + elems[2]] = myKey
            for ele in elems:
                it = self.tableWidget4.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget4.setItem(myKey, column, it)# can be done in dictionary of dictionary way, better
                it.setText(ele)
                column+=1
            self.tableWidget4.selectRow(myKey)
        else:
            self.tableWidget1.selectRow(row)

# main

In [16]:
if __name__ == "__main__":
    AcctNameDict = GetAcctName()#pd.read_csv("./depend/accountMap.txt")
    AcctNameDict[0] = "UNKNOWN"
    print(AcctNameDict.keys())
    app = QtWidgets.QApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    w = Control_sys_Tab()
    w.show()
    sys.exit(app.exec_())

dict_keys([8001, 8002, 8003, 8004, 8005, 8006, 8007, 8008, 8009, 8011, 8012, 8010, 8013, 8014, 8015, 8016, 8017, 0])


SystemExit: 0

C:\Users\taosu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# UI_control_system

In [12]:
class Ui_Control_system(QTabWidget):
    

    def setupUi(self, Control_system):

        Control_system.setObjectName("Control_system")
        Control_system.resize(1800, 985)
        #Control_system.setFixedSize(1004, 722)

        Control_system.setWindowIcon(QtGui.QIcon("images/icon.png"))
        Control_system.setStyleSheet(
            "QMainWindow{border-image:url(images/background.jpg)}"
        )
        
        
        self.centralwidget = QtWidgets.QWidget(Control_system)
        self.centralwidget.setObjectName("centralwidget")

        # 表格显示

        self.tableWidget1 = QtWidgets.QTableWidget(self.centralwidget)
        self.tableWidget1.setGeometry(QtCore.QRect(10, 10, 925, 910))
        self.tableWidget1.setRowCount(5000)
        self.tableWidget1.setColumnCount(11)
        self.tableWidget1.setObjectName("tableWidget")
        Control_system.setCentralWidget(self.centralwidget)
        self.tableWidget1.setAutoFillBackground(True)
        self.tableWidget1.setHorizontalHeaderLabels(["Acct","Instrument","BatchID","RQID","Direction","OrderSize","TradedVol","AvgPrice","Notional","FillRate","RefPrice"])
        for i in range(0,11):
            self.tableWidget1.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
        #self.tableWidget.item(0, 0).setFont(font)

        self.tableWidget2 = QtWidgets.QTableWidget(self.centralwidget)
        self.tableWidget2.setGeometry(QtCore.QRect(945, 10, 960, 450))
        self.tableWidget2.setRowCount(20000)
        self.tableWidget2.setColumnCount(14)
        self.tableWidget2.setObjectName("tableWidget")
        Control_system.setCentralWidget(self.centralwidget)
        self.tableWidget2.setAutoFillBackground(True)
        self.tableWidget2.setHorizontalHeaderLabels(["OrderRef", "RequestID", "PriceType","Direction","OffsetFlag","HedgeFlag","LimitPrice","VolOriginal","VolRemain","VolTraded","VolConfirmed","Status","OrderSysID","ExchID"])
        for i in range(0,14):
            self.tableWidget2.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
        #self.tableWidget.item(0, 0).setFont(font)

        self.tableWidget3 = QtWidgets.QTableWidget(self.centralwidget)
        #self.tableWidget3.setTextAlignment(Qt.AlignHCenter)
        self.tableWidget3.setGeometry(QtCore.QRect(945, 470, 960, 450))
        self.tableWidget3.setRowCount(500)
        self.tableWidget3.setColumnCount(6)
        #self.tableWidget3.setStyleSheet('background-repeat:repeat;')  #font color
        #self.tableWidget3.setStyleSheet('color:darkblue;')  #font color
        #self.tableWidget3.setStyleSheet('text-align:center;')
        #self.tableWidget3.setStyleSheet('vertical-align:super;')
        #self.tableWidget3.setStyleSheet('background-color:lightblue')
        #self.tableWidget3.horizontalHeader().setStyleSheet('QHeaderView::section{background:gray}')

        self.tableWidget3.setObjectName("tableWidget")
        Control_system.setCentralWidget(self.centralwidget)
        self.tableWidget3.setAutoFillBackground(True)
        self.tableWidget3.setHorizontalHeaderLabels(["BatchID", "AcctName","BuyNotional", "SellNotional","BuyFillRate","SellFillRate"])
        for i in range(0,6):
            self.tableWidget3.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            #self.tableWidget3.
        self.tableWidget3.setColumnWidth(0,150)
        self.tableWidget3.setColumnWidth(1,150)
        self.tableWidget3.setColumnWidth(2,100)
        self.tableWidget3.setColumnWidth(3,100)
        self.tableWidget3.setColumnWidth(4,100)
        self.tableWidget3.setColumnWidth(5,100)
        #self.label = QtWidgets.QLabel(self.centralwidget)
        #self.label.setGeometry(QtCore.QRect(360, 70, 300, 50))
        #self.label.setObjectName("label")
        #self.label.setAutoFillBackground(True)
        #self.label.setAlignment(QtCore.Qt.AlignCenter)
        #self.label.setStyleSheet("border-image:url(images/title.png)")

        '''
        self.tableWidget4 = QtWidgets.QTableWidget(self.centralwidget)
        self.tableWidget4.setGeometry(QtCore.QRect(1450, 470, 450, 450))
        self.tableWidget4.setRowCount(5000)
        self.tableWidget4.setColumnCount(4)
        self.tableWidget4.setObjectName("tableWidget")
        Control_system.setCentralWidget(self.centralwidget)
        self.tableWidget4.setAutoFillBackground(True)
        self.tableWidget4.setHorizontalHeaderLabels(["AcctID","Instrument", "OrderRef", "ErrorMsg"])
        for i in range(0,4):
            self.tableWidget4.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
        #self.tableWidget.item(0, 0).setFont(font)
        '''
		
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(1000, 925, 121, 41))
        self.pushButton.setObjectName("pushButton")

        Control_system.setCentralWidget(self.centralwidget)
        self.statusbar = QtWidgets.QStatusBar(Control_system)
        self.statusbar.setObjectName("statusbar")
        Control_system.setStatusBar(self.statusbar)

        self.retranslateUi(Control_system)
        QtCore.QMetaObject.connectSlotsByName(Control_system)

    def retranslateUi(self, Control_system):
        _translate = QtCore.QCoreApplication.translate
        Control_system.setWindowTitle(_translate("Control_system", "实时监控系统"))
        self.pushButton.setText(_translate("Control_system", "开始运行"))


## Control_system类

In [13]:
class Control_system(QtWidgets.QMainWindow, Ui_Control_system):

    def __init__(self, parent=None):
        super(Control_system, self).__init__(parent)
        self.RequestRowKey ={}
        self.OrderRowKey  ={}
        self.BatchRowKey  ={}   #To manage batch row index
        self.ErrorRowKey  ={}   #To manage error row index
        self.BuyBatchValue = {}
        self.SellBatchValue= {}
        self.BatchManagers  ={} # To manage value for each batch
        self.g_CurrRequestRow = 0
        self.g_CurrOrderRow  = 0
        self.g_CurrBatchRow  = 0
        self.g_CurrErrorRow  = 0
        super().__init__(parent)
        self.setupUi(self)
        self.pushButton.clicked.connect(self.slotStart)

    @QtCore.pyqtSlot()
    def slotStart(self):
        self.pushButton.setEnabled(False)
        self.update_data_thread = UpdateData(self)
        self.update_data_thread.requestChanged.connect(self.onRequestChanged)
        self.update_data_thread.start()

    @QtCore.pyqtSlot(int, int, str)
    def onRequestChanged(self, row, msgType, text):
        elems = text.split(',')
        #row =0
        column = 0
        if(msgType == 1):
            myKey = self.RequestRowKey.get(elems[2] + elems[3])
            if( myKey== None):
                myKey = self.g_CurrRequestRow
                self.g_CurrRequestRow+=1
                self.RequestRowKey[elems[2] + elems[3]] = myKey           ### CREATE A REQUEST ROW

            for ele in elems:
                it = self.tableWidget1.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget1.setItem(myKey, column, it)
                if(column==0):
                    it.setText(AcctNameDict[int(ele)])
                    #print("What is this: %d, and acctName =  %s"%(int(ele),AcctNameDict[int(ele)]))
                elif(column==4):
                    it.setText(directionType[ele])
                else:
                    it.setText(ele)
                column+=1
            self.tableWidget1.selectRow(myKey)

            myKey = self.BatchRowKey.get(elems[2])
            if( myKey== None):### UPDATE ACCT-BATCH TABLE HERE
                myKey = self.g_CurrBatchRow
                self.g_CurrBatchRow+=1
                self.BatchRowKey[elems[2]] = myKey
                self.BatchManagers[elems[2]] = BatchManager()                        ### CREATE A BATCH MANAGER
                self.BatchManagers[elems[2]].bookAcctID(int(elems[0]))
            #print("what is the direction code: %s"%(elems[4]))
            myTradeDirection = 1.0 if (int(elems[4]) == 0) else -1.0
            self.BatchManagers[elems[2]].bookTotalValue(elems[3],float(elems[5])*float(elems[10])*myTradeDirection)
            self.BatchManagers[elems[2]].bookTradedValue(elems[3],float(elems[8]))   ### need to change here
            print(elems[3],float(elems[8]))

            ### FILL INFORMATION IN THIS ROW
            ###print("GET NOTIONA:",str(self.BatchManagers[elems[2]].getBuyNotional()),str(self.BatchManagers[elems[2]].getSellNotional()))
            thisAcctID = self.BatchManagers[elems[2]].getAcctID()
            thisAcctIDStr = AcctNameDict[thisAcctID]
            vals1      = self.BatchManagers[elems[2]].getBuyNotional()
            vals2      = self.BatchManagers[elems[2]].getSellNotional()
            vals       = [elems[2],thisAcctIDStr,"{:,.2f}".format(vals1[0]),"{:,.2f}".format(vals2[0]),"{:.2%}".format(vals1[1]),"{:.2%}".format(vals2[1])]
            for column in range(0,6):
                it = self.tableWidget3.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget3.setItem(myKey, column, it)
                it.setText(vals[column])
            self.tableWidget3.selectRow(myKey)

        elif(msgType == 2):
            myKey = self.OrderRowKey.get(elems[0] + elems[1] + elems[12])
            if( myKey== None):
                myKey = self.g_CurrOrderRow
                self.g_CurrOrderRow+=1
                self.OrderRowKey[elems[0] + elems[1] + elems[12]] = myKey
            for ele in elems:
                it = self.tableWidget2.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget2.setItem(myKey, column, it)# can be done in dictionary of dictionary way, better
                if(column==3):
                    it.setText(directionType[ele])
                elif(column==4):
                    it.setText(OpenCloseFlag[ele])
                elif(column==5):
                    it.setText(hedgeFlag[ele])
                elif(column==11):
                    it.setText(orderStatus[ele])
                else:
                    it.setText(ele)
                column+=1
            self.tableWidget2.selectRow(myKey)

        elif(msgType == 3):
            myKey = self.ErrorRowKey.get(elems[0] + elems[2])
            if( myKey== None):
                myKey = self.g_CurrErrorRow
                self.g_CurrErrorRow += 1
                self.ErrorRowKey[elems[0] + elems[2]] = myKey
            for ele in elems:
                it = self.tableWidget4.item(myKey, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget4.setItem(myKey, column, it)# can be done in dictionary of dictionary way, better
                it.setText(ele)
                column+=1
            self.tableWidget4.selectRow(myKey)
        else:
            self.tableWidget1.selectRow(row)